#### Clean Data Notebook

1. Rename the columns
2. Irrelevant columns
3. Duplicates
4. Null Values in relevant columns
5. Outliers
6. Map column values
7. Visualise and investigate

In [4]:
import pandas as pd

In [5]:
column_names = ['id', 'price', 'date', 'postcode', 
        'type', 'new_build', 'land', 'primary_address', 
        'secondary_address', 'street', 'locality', 'town_city', 
        'district', 'county', 'ppd', 'record' ]

df = pd.read_csv("../data/raw/pp-complete.csv", names = column_names, parse_dates = ['date'])

In [7]:
df.head(5)

,id,price,date,postcode,type,new_build,land,primary_address,secondary_address,street,locality,town_city,district,county,ppd,record
0,{A42E2F04-2538-4A25-94C5-49E29C6C8FA8},18500,1995-01-31,TQ1 1RY,F,N,L,VILLA PARADISO,FLAT 10,HIGHER WARBERRY ROAD,TORQUAY,TORQUAY,TORBAY,TORBAY,A,A
1,{1BA349E3-2579-40D6-999E-49E2A25D2284},73450,1995-10-09,L26 7XJ,D,Y,F,6,NaN,CATKIN ROAD,LIVERPOOL,LIVERPOOL,KNOWSLEY,MERSEYSIDE,A,A
2,{E5B50DCB-BC7A-4E54-B167-49E2A6B4148B},59000,1995-03-31,BH12 2AE,D,N,F,28,NaN,ALDER ROAD,POOLE,POOLE,POOLE,POOLE,A,A
3,{81E50116-D675-4B7F-9F8D-49E2B5D43271},31000,1995-12-04,IP13 0DR,D,Y,F,NONSUCH COTTAGE,NaN,THE STREET,HACHESTON,WOODBRIDGE,SUFFOLK COASTAL,SUFFOLK,A,A
4,{B97455B9-75CB-40BB-A615-42C53683E143},95000,1995-09-22,WS14 0BE,D,N,F,FOX COVER COTTAGE,NaN,HALL LANE,LICHFIELD,LICHFIELD,LICHFIELD,STAFFORDSHIRE,A,A


In [8]:
print(f"The data contains {df.shape[0]} rows.")

The data contains 25530306 rows.


In [9]:
df.dtypes

id                           object
price                         int64
date                 datetime64[ns]
postcode                     object
type                         object
new_build                    object
land                         object
primary_address              object
secondary_address            object
street                       object
locality                     object
town_city                    object
district                     object
county                       object
ppd                          object
record                       object
dtype: object

In [10]:
# Since we know the id's are not duplicated we can search excluding the id to see if any houses are entered twice.
duplicates = df.loc[:, 'price':'record'][df.loc[:, 'price':'record'].duplicated()]
print(f"The number of duplicates excluding id is {duplicates.shape[0]}")

The number of duplicates excluding id is 12452


In [11]:
#Lets perfom a quick check to verify there are actually duplicates (The answer should be two observations)
df[(df['postcode']=='N17 6YD') & (df['price']==12500)]

,id,price,date,postcode,type,new_build,land,primary_address,secondary_address,street,locality,town_city,district,county,ppd,record
90383,{2A289E9C-E447-CDC8-E050-A8C063054829},12500,1995-02-06,N17 6YD,F,N,L,13,NaN,EVE ROAD,NaN,LONDON,HARINGEY,GREATER LONDON,A,A
90384,{2A289E9C-E448-CDC8-E050-A8C063054829},12500,1995-02-06,N17 6YD,F,N,L,13,NaN,EVE ROAD,NaN,LONDON,HARINGEY,GREATER LONDON,A,A


In [12]:
# Lets drop the duplicates, keeping only the first instance. 
df_no_duplicates = df.drop_duplicates(subset = df.columns[1:] , keep = "first")

In [13]:
#Lets check the number of empty rows in each column. 
df_no_duplicates.isnull().sum()

id                          0
price                       0
date                        0
postcode                40045
type                        0
new_build                   0
land                        0
primary_address          4191
secondary_address    22539865
street                 394033
locality              8380064
town_city                   0
district                    0
county                      0
ppd                         0
record                      0
dtype: int64

In [14]:
df_empty_postcodes = df_no_duplicates[df_no_duplicates['postcode'].isnull()]

In [15]:
df_empty_postcodes.head()

,id,price,date,postcode,type,new_build,land,primary_address,secondary_address,street,locality,town_city,district,county,ppd,record
1976,{BEAA2DC6-B692-408E-A2A3-3BAEDDFCB145},38950,1995-09-08,NaN,T,Y,L,66,NaN,NaN,MOSS HALL GROVE,LIVERPOOL,LIVERPOOL,MERSEYSIDE,A,A
3882,{8E9EC4A0-D0DD-4707-8FD8-87474D269417},60000,1995-10-16,NaN,D,N,F,3,NaN,PILGRIMS LANE,CHILHAM,CANTERBURY,ASHFORD,KENT,A,A
5186,{06B58F72-089A-49E9-9F0F-3F500D8FC30B},18500,1995-04-03,NaN,D,N,F,TAN YR ORSEDD,NaN,NaN,RHOSGOCH,RHOSGOCH,YNYS MON-ISLE OF ANGLESEY,GWYNEDD,A,A
6435,{51EA43A4-017B-4A59-987B-BDD1EA19FE8E},325000,1995-06-23,NaN,D,N,F,"THE GRANGE, 9",NaN,ORLEY FARM ROAD,HARROW,HARROW,HARROW,GREATER LONDON,A,A
9123,{51F279F5-EF5F-46E1-BD8E-B6C4159D8FA7},34000,1995-01-03,NaN,D,N,F,PARNALL ROAD INDUSTRIAL ESTATE,UNIT 4,PARNALL ROAD,BRISTOL,BRISTOL,BRISTOL,AVON,A,A


In [16]:
#Difficult decision - we can obviously try and workout a way to get the postcode using the roads but 
#that can prove to be too much work for us
#For now let us delete the observations with no postcode. 
df_complete = df_no_duplicates[df_no_duplicates['postcode'].notnull()]

In [17]:
df_complete.shape[0]

25477809

In [18]:
#Lets view the range of the data (10 largest and 10 smallest)
df_complete.nlargest(n=10, columns='price')

,id,price,date,postcode,type,new_build,land,primary_address,secondary_address,street,locality,town_city,district,county,ppd,record
23109194,{582D0637-EE28-8F22-E053-6C04A8C01BAC},594300000,2017-07-31,W1U 8EW,O,N,L,55,UNIT 53,BAKER STREET,NaN,LONDON,CITY OF WESTMINSTER,GREATER LONDON,B,A
24152317,{666758D7-A512-3363-E053-6B04A8C0D74E},569200000,2018-02-08,W1J 7BT,O,N,F,2,NaN,STANHOPE ROW,NaN,LONDON,CITY OF WESTMINSTER,GREATER LONDON,B,A
23286231,{7E86B6FB-BF88-458C-E053-6B04A8C0C84C},448500000,2018-09-21,SE1 9JZ,O,N,F,22,NaN,SUMNER STREET,NaN,LONDON,SOUTHWARK,GREATER LONDON,B,A
23941390,{773788C3-A146-2CE4-E053-6C04A8C05E57},448300979,2018-08-17,NE6 1AS,O,N,L,UNIT 8,NaN,HAWICK CRESCENT INDUSTRIAL ESTATE,NaN,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE,TYNE AND WEAR,B,A
23907865,{6DA0844A-CDA7-30F2-E053-6B04A8C05F3B},415000000,2018-02-28,WS1 1RY,O,N,L,37 - 39,NaN,DIGBETH,NaN,WALSALL,WALSALL,WEST MIDLANDS,B,A
23907867,{6DA0844A-CDA9-30F2-E053-6B04A8C05F3B},415000000,2018-02-28,WS1 1RY,O,N,F,37,NaN,DIGBETH,NaN,WALSALL,WALSALL,WEST MIDLANDS,B,A
24920018,{8CAC1318-F392-0253-E053-6B04A8C08E51},411500000,2019-04-04,EC2V 7PG,O,N,L,30,"FIRST, SEVENTH AND EIGHTH FLOORS",GRESHAM STREET,NaN,LONDON,CITY OF LONDON,GREATER LONDON,B,A
21860913,{55BDCAE6-DC9C-521D-E053-6B04A8C0DD7A},370000000,2016-10-31,SW1H 0BG,O,N,F,NEW SCOTLAND YARD 8-10,NaN,BROADWAY,NaN,LONDON,CITY OF WESTMINSTER,GREATER LONDON,B,A
24324476,{98C75472-BCE7-72E9-E053-6B04A8C042F0},337000000,2019-04-04,EC4V 5AE,O,N,F,35 - 45,NaN,CARTER LANE,NaN,LONDON,CITY OF LONDON,GREATER LONDON,B,A
21862501,{55BDCAE6-C6B5-521D-E053-6B04A8C0DD7A},330000000,2016-10-20,TW6 2GD,O,N,L,TERMINAL 5,SOFITEL,WENTWORTH DRIVE,LONDON HEATHROW AIRPORT,HOUNSLOW,HILLINGDON,GREATER LONDON,B,A


I think we can all admit - we can't afford these kind of houses. I really wanted to make this project for a normal ordinary house. So what I really want to do is to cap the data to just houses that are less than a million. 

In [19]:
# Lets View the cheaperst houses. 
df_complete.nsmallest(n=10, columns='price')

,id,price,date,postcode,type,new_build,land,primary_address,secondary_address,street,locality,town_city,district,county,ppd,record
95948,{2A289E9D-31D2-CDC8-E050-A8C063054829},1,1995-04-18,EX34 9JU,T,N,F,5,NaN,APSLEY TERRACE,NaN,ILFRACOMBE,NORTH DEVON,DEVON,A,A
490887,{2A289E9F-B709-CDC8-E050-A8C063054829},1,1995-08-04,M30 9AF,T,N,F,8,NaN,DOUGHTY AVENUE,ECCLES,MANCHESTER,SALFORD,GREATER MANCHESTER,A,A
520198,{2A289E9E-EB2B-CDC8-E050-A8C063054829},1,1995-09-11,KT1 4DQ,T,N,F,98,NaN,HIGH STREET,HAMPTON WICK,KINGSTON UPON THAMES,RICHMOND UPON THAMES,GREATER LONDON,A,A
523780,{2A289E9E-AEB6-CDC8-E050-A8C063054829},1,1995-03-17,W3 9RN,S,N,F,3,NaN,HILLCREST ROAD,ACTON,LONDON,EALING,GREATER LONDON,A,A
524227,{2A289E9E-EE9B-CDC8-E050-A8C063054829},1,1995-12-08,SG5 4LW,S,N,F,34,NaN,UPPERSTONE CLOSE,STOTFOLD,HITCHIN,CENTRAL BEDFORDSHIRE,CENTRAL BEDFORDSHIRE,A,A
552204,{2A289E9F-77B4-CDC8-E050-A8C063054829},1,1995-12-08,SN12 7RR,S,N,F,19,NaN,MALVERN CLOSE,NaN,MELKSHAM,WILTSHIRE,WILTSHIRE,A,A
566664,{2A289E9D-D2C7-CDC8-E050-A8C063054829},1,1995-05-12,OX14 1EL,S,N,L,19,NaN,FITZHARRYS ROAD,NaN,ABINGDON,VALE OF WHITE HORSE,OXFORDSHIRE,A,A
568492,{2A289E9D-E7BD-CDC8-E050-A8C063054829},1,1995-02-10,SW11 1HT,T,N,F,16A,NaN,CHIVALRY ROAD,NaN,LONDON,WANDSWORTH,GREATER LONDON,A,A
572479,{2A289E9D-EB15-CDC8-E050-A8C063054829},1,1995-01-13,BA21 5TQ,S,N,F,1,NaN,HILL VIEW,MUDFORD,YEOVIL,SOUTH SOMERSET,SOMERSET,A,A
579188,{2A289E9D-1927-CDC8-E050-A8C063054829},1,1995-01-12,TQ1 4BE,T,N,F,147,NaN,LYMINGTON ROAD,NaN,TORQUAY,TORBAY,TORBAY,A,A


In [20]:
# From this I think we should only observe houses that are between 10,000 and 1,000,000
df_price_cut = df_complete[(df_complete['price']<=1000000) & (df['price']>=10000)].reset_index(drop = True)

<ipython-input-20-3d3e9660becf>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_price_cut = df_complete[(df_complete['price']<=1000000) & (df['price']>=10000)].reset_index(drop = True)


In [27]:
perc_reduction = ((df_complete.shape[0] - df_price_cut.shape[0])/df_complete.shape[0])*100
print(f"filtered out {perc_reduction} % of data")

filtered out 1.0750374963561427 % of data


In [28]:
df_price_cut.to_csv("../data/processed/house_1_price_cut.csv", index = False)

In [29]:
# Going forward we need to add some columns

In [31]:
new_build = {'Y': 1,'N': 0}
df_price_cut.new_build = [new_build[item] for item in df_price_cut.new_build]
df_price_cut['year'] = df_price_cut['date'].dt.year
df_price_cut['month'] = df_price_cut['date'].dt.year
df_price_cut['month_year'] = df_price_cut['date'].astype('datetime64[M]')

In [32]:
df_price_cut.head()

,id,price,date,postcode,type,new_build,land,primary_address,secondary_address,street,locality,town_city,district,county,ppd,record,year,month,month_year
0,{A42E2F04-2538-4A25-94C5-49E29C6C8FA8},18500,1995-01-31,TQ1 1RY,F,0,L,VILLA PARADISO,FLAT 10,HIGHER WARBERRY ROAD,TORQUAY,TORQUAY,TORBAY,TORBAY,A,A,1995,1995,1995-01-01
1,{1BA349E3-2579-40D6-999E-49E2A25D2284},73450,1995-10-09,L26 7XJ,D,1,F,6,NaN,CATKIN ROAD,LIVERPOOL,LIVERPOOL,KNOWSLEY,MERSEYSIDE,A,A,1995,1995,1995-10-01
2,{E5B50DCB-BC7A-4E54-B167-49E2A6B4148B},59000,1995-03-31,BH12 2AE,D,0,F,28,NaN,ALDER ROAD,POOLE,POOLE,POOLE,POOLE,A,A,1995,1995,1995-03-01
3,{81E50116-D675-4B7F-9F8D-49E2B5D43271},31000,1995-12-04,IP13 0DR,D,1,F,NONSUCH COTTAGE,NaN,THE STREET,HACHESTON,WOODBRIDGE,SUFFOLK COASTAL,SUFFOLK,A,A,1995,1995,1995-12-01
4,{B97455B9-75CB-40BB-A615-42C53683E143},95000,1995-09-22,WS14 0BE,D,0,F,FOX COVER COTTAGE,NaN,HALL LANE,LICHFIELD,LICHFIELD,LICHFIELD,STAFFORDSHIRE,A,A,1995,1995,1995-09-01


In [35]:
#Adjust price to inflation
df_multiplier = pd.read_csv("../data/raw/inflation_multiplier.csv", usecols = ['Year', 'Multiplier'], dtype = {'Year': 'int64', 'Multiplier': 'float64'})
df_multiplier.columns =['year', 'multiplier']

In [36]:
df_multiplier.dtypes

year            int64
multiplier    float64
dtype: object

In [38]:
df_house_data = pd.merge(df_price_cut, df_multiplier, on='year')

In [39]:
df_house_data['adjusted_price'] = df_house_data['price'] * df_house_data['multiplier']

In [40]:
df_house_data.head()

,id,price,date,postcode,type,new_build,land,primary_address,secondary_address,street,...,town_city,district,county,ppd,record,year,month,month_year,multiplier,adjusted_price
0,{A42E2F04-2538-4A25-94C5-49E29C6C8FA8},18500,1995-01-31,TQ1 1RY,F,0,L,VILLA PARADISO,FLAT 10,HIGHER WARBERRY ROAD,...,TORQUAY,TORBAY,TORBAY,A,A,1995,1995,1995-01-01,2.01,37185.0
1,{1BA349E3-2579-40D6-999E-49E2A25D2284},73450,1995-10-09,L26 7XJ,D,1,F,6,NaN,CATKIN ROAD,...,LIVERPOOL,KNOWSLEY,MERSEYSIDE,A,A,1995,1995,1995-10-01,2.01,147634.5
2,{E5B50DCB-BC7A-4E54-B167-49E2A6B4148B},59000,1995-03-31,BH12 2AE,D,0,F,28,NaN,ALDER ROAD,...,POOLE,POOLE,POOLE,A,A,1995,1995,1995-03-01,2.01,118590.0
3,{81E50116-D675-4B7F-9F8D-49E2B5D43271},31000,1995-12-04,IP13 0DR,D,1,F,NONSUCH COTTAGE,NaN,THE STREET,...,WOODBRIDGE,SUFFOLK COASTAL,SUFFOLK,A,A,1995,1995,1995-12-01,2.01,62310.0
4,{B97455B9-75CB-40BB-A615-42C53683E143},95000,1995-09-22,WS14 0BE,D,0,F,FOX COVER COTTAGE,NaN,HALL LANE,...,LICHFIELD,LICHFIELD,STAFFORDSHIRE,A,A,1995,1995,1995-09-01,2.01,190950.0


In [41]:
df_house_data.nlargest(n=10, columns='adjusted_price')

,id,price,date,postcode,type,new_build,land,primary_address,secondary_address,street,...,town_city,district,county,ppd,record,year,month,month_year,multiplier,adjusted_price
66420,{48F1260A-F447-491D-B627-C33F9E1D5010},1000000,1995-06-27,SW10 9SP,T,0,F,9,NaN,PRIORY WALK,...,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A,1995,1995,1995-06-01,2.01,2010000.0
148663,{41FC4910-C5D6-4AC3-B073-C58680CC51F4},1000000,1995-09-08,W8 6PR,S,0,F,18,NaN,SCARSDALE VILLAS,...,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A,1995,1995,1995-09-01,2.01,2010000.0
227300,{D7A13F02-6FBD-439E-8404-169FE5808583},1000000,1995-12-07,SW7 2NP,T,0,L,4,NaN,PELHAM CRESCENT,...,LONDON,KENSINGTON AND CHELSEA,GREATER LONDON,A,A,1995,1995,1995-12-01,2.01,2010000.0
259581,{5F56B504-1472-46EA-9F72-B57E636EF71C},1000000,1995-09-15,NW11 6XB,F,0,L,34,NaN,WILDWOOD ROAD,...,LONDON,BARNET,GREATER LONDON,A,A,1995,1995,1995-09-01,2.01,2010000.0
304459,{100CE525-FB27-43F5-9779-308446676973},1000000,1995-05-26,SL6 2NA,D,0,F,CHUFFS HOUSE,NaN,MONEYROW GREEN,...,MAIDENHEAD,WINDSOR AND MAIDENHEAD,WINDSOR AND MAIDENHEAD,A,A,1995,1995,1995-05-01,2.01,2010000.0
337083,{3588B03D-6A99-478F-B75B-62E89BDF0593},1000000,1995-06-08,NW8 9DX,T,0,F,21,NaN,ABERCORN PLACE,...,LONDON,CITY OF WESTMINSTER,GREATER LONDON,A,A,1995,1995,1995-06-01,2.01,2010000.0
355874,{31A6FB6C-9BE1-475D-8752-DA4FB10FA036},1000000,1995-08-30,NW8 9BW,F,1,L,20,FLAT 127,ABBEY ROAD,...,LONDON,CITY OF WESTMINSTER,GREATER LONDON,A,A,1995,1995,1995-08-01,2.01,2010000.0
473462,{EA97603D-9036-4813-AEBA-B0BB8A3E8E4E},1000000,1995-06-21,GU34 5NQ,D,0,F,HATTINGLEY HOUSE,NaN,HATTINGLEY ROAD,...,ALTON,EAST HAMPSHIRE,HAMPSHIRE,A,A,1995,1995,1995-06-01,2.01,2010000.0
521368,{F697DE00-EEEE-41C1-9E5E-38F71B30F195},1000000,1995-03-17,BR6 8LL,D,0,F,8,NaN,PARK AVENUE,...,ORPINGTON,BROMLEY,GREATER LONDON,A,A,1995,1995,1995-03-01,2.01,2010000.0
529507,{2A289EA1-877A-CDC8-E050-A8C063054829},1000000,1995-01-26,SW15 2BZ,D,0,F,4,NaN,CARLTON DRIVE,...,LONDON,WANDSWORTH,GREATER LONDON,A,A,1995,1995,1995-01-01,2.01,2010000.0


In [42]:
df_house_data.to_csv("../data/processed/house_2_adjusted_data.csv", index = False)

In [43]:
df_multiple_transactions = df_house_data.loc[:, 'postcode':'county'][df_house_data.loc[:, 'postcode':'county'].duplicated()]

In [44]:
df_multiple_transactions.shape

(6795597, 11)

In [45]:
# Lets drop the duplicates, keeping only the first instance. 
df_house_data_ordered = df_house_data.sort_values(by=['date'])
df_house_last_sale = df_house_data_ordered.drop_duplicates(subset = df_house_data_ordered.columns[3:14] ,keep = "last")

In [46]:
df_house_last_sale.shape

(18408316, 21)

In [48]:
df_house_last_sale.to_csv("../data/processed/house_3_last_sale.csv", index = False)